In [3]:
import math
import requests
import random
import bisect
import numpy as np
from geopy.geocoders import Nominatim
import folium
import matplotlib.pyplot as plt
import rasterio
from rasterio.io import MemoryFile
from geopy.geocoders import Nominatim
import os
import osmnx as ox
import shapely.geometry as geom
import tempfile
from shapely.geometry import MultiPoint

###############################################################################
# DEM-BASED PARAMETERS & DATA SETUP
################################################################################
# DEM_PATH = "/Users/chrisjuarez/moo_node/nasaDEM.tif"
GATEWAY_HEIGHT = 2.0
SENSOR_HEIGHT = 2.0
CLEARANCE_BUFFER = 2.9
NUM_LOS_SAMPLES = 500

#dem_dataset = rasterio.open(DEM_PATH)

###############################################################################
# 1) DOMAIN EXTRACTION USING GEOPY 
###############################################################################
geolocator = Nominatim(user_agent="MySensorPlacementApp/1.0 (your_email@example.com)", timeout=10)
location = geolocator.geocode("Crestline, California, USA", addressdetails=True)
if location is None:
    print("Cannot find location")
    exit()
lon, lat = float(location.longitude), float(location.latitude)
print(f"Found at centroid: ({lon}, {lat})")
if 'boundingbox' in location.raw:
    south, north, west, east = map(float, location.raw['boundingbox'])
    XMIN, XMAX = west, east
    YMIN, YMAX = south, north
else:
    delta = 0.05
    XMIN, XMAX = lon - delta, lon + delta
    YMIN, YMAX = lat - delta, lat + delta
print(f"Domain: X {XMIN}→{XMAX}, Y {YMIN}→{YMAX}")

# DEM data fetching from OpenTopography
OPENTOPO_API_KEY = '50a0cd794b0c3c7d0809733e4498ffef'
url = "https://portal.opentopography.org/API/globaldem"
params = {
    "demtype": "SRTMGL1",
    "south": YMIN,
    "north": YMAX,
    "west": XMIN,
    "east": XMAX,
    "outputFormat": "GTiff",
    "API_Key": OPENTOPO_API_KEY
}

response = requests.get(url, params=params)

if response.status_code == 200:
    with MemoryFile(response.content) as memfile:
        with memfile.open() as src:
            dem_array = src.read(1)
            dem_transform = src.transform
            dem_bounds = src.bounds
            dem_crs = src.crs
            dem_profile = src.profile

    # Now reopen it explicitly for consistent access later
    dem_dataset = rasterio.io.MemoryFile().open(**dem_profile)
    dem_dataset.write(dem_array, 1)

    print("DEM data loaded successfully from OpenTopography.")
else:
    raise Exception(f"Error fetching DEM data: {response.status_code}")

NUM_SENSORS = 7
print(f"Sensors (excl. gateway): {NUM_SENSORS}")

###############################################################################
# 2) COVERAGE & NSGA-II PARAMS
###############################################################################
SENSOR_COVERAGE_RADIUS_M   = 400
SENSOR_COVERAGE_RADIUS_DEG = SENSOR_COVERAGE_RADIUS_M / 111000.0
NUM_COVERAGE_POINTS        = 9000
WEIGHT_COVERAGE            = 7000.0
WEIGHT_LOS                 = 2.0
WEIGHT_POLYGON_AREA = 100.0 # Add a weight for the polygon area objective
WEIGHT_GATEWAY_CENTRALITY = 350.0 # Weight for central gateway placement

POP_SIZE   = 580
MAX_GEN    = 25
MUT_RATE   = 0.4
CROSS_RATE = 0.8
ALPHA_BLX  = 0.5
ETA_MUT    = 20.0

coverage_sample_points = [
    (random.uniform(XMIN, XMAX), random.uniform(YMIN, YMAX))
    for _ in range(NUM_COVERAGE_POINTS)
]

###############################################################################
# 3) BUILD WALKABLE GRAPH & ATTACH ELEVATION
###############################################################################
poly = geom.box(XMIN, YMIN, XMAX, YMAX)
G = ox.graph_from_polygon(poly, network_type='all_public')
# Save the in-memory raster to a temporary file
with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as tmp_dem_file:
    profile = dem_dataset.profile
    profile.update(driver='GTiff')
    with rasterio.open(tmp_dem_file.name, 'w', **profile) as dst:
        dst.write(dem_array, 1)
    temp_dem_path = tmp_dem_file.name

try:
    G = ox.elevation.add_node_elevations_raster(G, temp_dem_path, cpus=1)
finally:
    # Clean up the temporary file
    os.remove(temp_dem_path)

walkable_nodes = list(G.nodes(data=True))
walkable_coords = [
    (float(data['x']), float(data['y']), float(data['elevation']))
    for node_id, data in walkable_nodes
    if 'elevation' in data and not np.isnan(data['elevation'])
]

# ─── ELEVATION‐WEIGHTED SAMPLER ────────────────────────────────────────────────
# Extract elevations from your existing walkable nodes
elevations = np.array([e for _, _, e in walkable_coords])

# Define dynamic elevation threshold (e.g., 80th percentile)
ELEV_PERCENTILE = 50  # Adjust percentile as desired
ELEV_THRESH = np.percentile(elevations, ELEV_PERCENTILE)

print(f"Dynamically determined elevation threshold: {ELEV_THRESH:.2f} meters")

# Filter walkable coordinates based on dynamic threshold
walkable_coords = [
    (x, y, e) for x, y, e in walkable_coords if e >= ELEV_THRESH
]
def weighted_choice_by_elevation(coords, power=0.5):
    elevs   = np.array([e for _,_,e in coords])
    weights = (elevs - elevs.min() + 1e-3)**power
    weights = weights / weights.sum()
    cumw    = np.cumsum(weights)
    r       = random.random()
    idx     = bisect.bisect_left(cumw, r)
    return coords[idx]

# ─── UPDATED CHROMOSOME GENERATOR ────────────────────────────────────────────
def polygonal_chromosome():
    coords = []
    selected_points = set()
    attempts = 0
    max_attempts = 1000

    while len(selected_points) < NUM_SENSORS + 1 and attempts < max_attempts:
        candidate = weighted_choice_by_elevation(walkable_coords)
        point = (candidate[0], candidate[1])
        selected_points.add(point)
        attempts += 1

    if selected_points:  # Ensure there are selected points before calculating the hull
        points = MultiPoint(list(selected_points)).convex_hull.exterior.coords
        coords = [coord for point in points[:-1] for coord in point]  # ensure closed polygon excluding duplicate last point

        # Truncate or fill the coords to ensure exactly NUM_SENSORS+1 nodes
        target_nodes = NUM_SENSORS + 1
        current_nodes = len(coords) // 2

        if current_nodes > target_nodes:
            coords = coords[:2 * target_nodes]
        while current_nodes < target_nodes:
            extra = weighted_choice_by_elevation(walkable_coords)
            coords.extend([extra[0], extra[1]])
            current_nodes += 1
    else:
        # Fallback to a random chromosome if no points were selected
        print("Warning: Could not form initial polygon. Falling back to random placement.")
        coords = []
        gx, gy, _ = weighted_choice_by_elevation(walkable_coords)
        coords.extend([gx, gy])
        for _ in range(NUM_SENSORS):
            sx, sy, _ = weighted_choice_by_elevation(walkable_coords)
            coords.extend([sx, sy])

    return coords

###############################################################################
# 3) DEM-BASED LOS EVALUATION FUNCTION (FRESNEL ZONE ANALYSIS)
###############################################################################
def compute_dem_los_score(gateway_coord, sensor_coord,
                          dem_dataset, clearance_buffer=CLEARANCE_BUFFER,
                          sensor_height=SENSOR_HEIGHT, gateway_height=GATEWAY_HEIGHT,
                          num_samples=NUM_LOS_SAMPLES):
    """
    Compute a LOS quality score between a gateway and a sensor using DEM data and Fresnel zone clearance.
    Returns a score in [0, 1] (1 is unobstructed) and 0 if out-of-bounds.
    """
    bounds = dem_dataset.bounds  # (left, bottom, right, top)
    if not (bounds.left <= gateway_coord[0] <= bounds.right and bounds.bottom <= gateway_coord[1] <= bounds.top):
        return 0.0
    if not (bounds.left <= sensor_coord[0] <= bounds.right and bounds.bottom <= sensor_coord[1] <= bounds.top):
        return 0.0

    gw_row, gw_col = dem_dataset.index(gateway_coord[0], gateway_coord[1])
    gw_row = min(max(gw_row, 0), dem_dataset.height - 1)
    gw_col = min(max(gw_col, 0), dem_dataset.width - 1)
    
    sensor_row, sensor_col = dem_dataset.index(sensor_coord[0], sensor_coord[1])
    sensor_row = min(max(sensor_row, 0), dem_dataset.height - 1)
    sensor_col = min(max(sensor_col, 0), dem_dataset.width - 1)
    
    gw_ground_elev = dem_dataset.read(1)[gw_row, gw_col]
    sensor_ground_elev = dem_dataset.read(1)[sensor_row, sensor_col]
    
    gw_effective = gw_ground_elev + gateway_height
    sensor_effective = sensor_ground_elev + sensor_height

    xs = np.linspace(gateway_coord[0], sensor_coord[0], num_samples)
    ys = np.linspace(gateway_coord[1], sensor_coord[1], num_samples)
    los_line_heights = np.linspace(gw_effective, sensor_effective, num_samples)
    
    ground_elevations = []
    for x, y in zip(xs, ys):
        row, col = dem_dataset.index(x, y)
        row = min(max(row, 0), dem_dataset.height - 1)
        col = min(max(col, 0), dem_dataset.width - 1)
        ground_elevations.append(dem_dataset.read(1)[row, col])
    ground_elevations = np.array(ground_elevations)
    
    clearances = los_line_heights - (ground_elevations + clearance_buffer)
    penalty = np.sum(np.abs(np.minimum(clearances, 0)))
    score = math.exp(-penalty)
    return score

###############################################################################
# 4) OBJECTIVE FUNCTIONS (EVALUATION) + CACHING
###############################################################################
eval_cache = {}

GRID_SIZE = 85  # Adjust for coverage grid resolution
x_grid = np.linspace(XMIN, XMAX, GRID_SIZE)
y_grid = np.linspace(YMIN, YMAX, GRID_SIZE)
coverage_sample_points = [(x, y) for x in x_grid for y in y_grid]


def get_elevation(coord, dem_dataset):
    try:
        row, col = dem_dataset.index(coord[0], coord[1])
        row = min(max(row, 0), dem_dataset.height - 1)
        col = min(max(col, 0), dem_dataset.width - 1)
        return dem_dataset.read(1)[row, col]
    except rasterio.RasterioIOError:
        return None


def evaluate(chrom):
    gx, gy = chrom[0], chrom[1]
    sensor_coords = [(chrom[i], chrom[i+1]) for i in range(2, len(chrom), 2)]
    num_sensors = len(sensor_coords)

    # Overlap penalty: strongly discourage sensor overlap
    overlap_penalty = sum(
        max(0, (2 * SENSOR_COVERAGE_RADIUS_DEG) - math.dist(sensor_coords[i], sensor_coords[j]))
        for i in range(len(sensor_coords)) for j in range(i + 1, len(sensor_coords))
    ) * 2000.0  # Stronger penalty factor

    # Gateway centrality penalty
    if num_sensors > 0:
        avg_sx = sum(s[0] for s in sensor_coords) / num_sensors
        avg_sy = sum(s[1] for s in sensor_coords) / num_sensors
        centroid_sensors = (avg_sx, avg_sy)
        gateway_centrality_penalty = math.dist((gx, gy), centroid_sensors) * WEIGHT_GATEWAY_CENTRALITY
    else:
        gateway_centrality_penalty = 0.0

    # Explicit coverage calculation using structured grid
    covered_points = set()
    for idx, pt in enumerate(coverage_sample_points):
        for sensor in sensor_coords:
            if math.dist(pt, sensor) <= SENSOR_COVERAGE_RADIUS_DEG:
                covered_points.add(idx)
                break
    coverage_metric = len(covered_points) / len(coverage_sample_points)

    # Penalty for uncovered points to ensure widespread sensor distribution
    uncovered_penalty = (1 - coverage_metric) * 4000.0  # Stronger penalty for incomplete coverage

    # Cost calculation combines distance, overlap, uncovered penalties, and gateway centrality
    base_cost = 100.0 * NUM_SENSORS
    distance_cost = sum(math.dist((gx, gy), sensor) for sensor in sensor_coords)
    cost = base_cost + distance_cost + overlap_penalty + gateway_centrality_penalty + uncovered_penalty

    # Polygon coverage area to encourage spatial distribution
    all_coords = sensor_coords + [(gx, gy)]
    poly_area = MultiPoint(all_coords).convex_hull.area if len(all_coords) >= 3 else 0.0

    # LOS (elevation variance minimization)
    elevs = [get_elevation(coord, dem_dataset) for coord in all_coords if get_elevation(coord, dem_dataset) is not None]
    los_variance = np.var(elevs) if elevs else 0.0

    return cost, -WEIGHT_COVERAGE * coverage_metric, -WEIGHT_POLYGON_AREA * poly_area, WEIGHT_LOS * los_variance

# Update the evaluate_cached function if you are using it
def evaluate_cached(chrom):
    key = tuple(round(v, 6) for v in chrom)
    if key not in eval_cache:
        eval_cache[key] = evaluate(chrom)
    return eval_cache[key]
###############################################################################
# 5) NSGA-II UTILITY FUNCTIONS
###############################################################################
def dominates(objA, objB):
    better_or_equal = True
    strictly_better = False
    for a, b in zip(objA, objB):
        if a > b:
            better_or_equal = False
            break
    if not better_or_equal:
        return False
    for a, b in zip(objA, objB):
        if a < b:
            strictly_better = True
            break
    return strictly_better

def fast_non_dominated_sort(pop_objs):
    pop_size = len(pop_objs)
    S = [[] for _ in range(pop_size)]
    n = [0] * pop_size
    rank = [0] * pop_size

    for i in range(pop_size):
        S[i] = []
        n[i] = 0
    for i in range(pop_objs.__len__()):
        for j in range(pop_size):
            if i == j:
                continue
            if dominates(pop_objs[i], pop_objs[j]):
                S[i].append(j)
            elif dominates(pop_objs[j], pop_objs[i]):
                n[i] += 1

    front = [[]]
    for i in range(pop_size):
        if n[i] == 0:
            rank[i] = 0
            front[0].append(i)

    k = 0
    while front[k]:
        Q = []
        for i in front[k]:
            for j in S[i]:
                n[j] -= 1
                if n[j] == 0:
                    rank[j] = k + 1
                    Q.append(j)
        k += 1
        front.append(Q)
    return rank

def crowding_distance(obj_vals, front_indices):
    distances = {idx: 0.0 for idx in front_indices}
    if len(front_indices) <= 2:
        for idx in front_indices:
            distances[idx] = float('inf')
        return distances

    num_obj = len(obj_vals[0])
    for m in range(num_obj):
        sorted_indices = sorted(front_indices, key=lambda idx: obj_vals[idx][m])
        min_val = obj_vals[sorted_indices[0]][m]
        max_val = obj_vals[sorted_indices[-1]][m]
        distances[sorted_indices[0]] = float('inf')
        distances[sorted_indices[-1]] = float('inf')
        if abs(max_val - min_val) < 1e-9:
            continue
        for i in range(1, len(sorted_indices)-1):
            idx = sorted_indices[i]
            distances[idx] += (obj_vals[sorted_indices[i+1]][m] - obj_vals[sorted_indices[i-1]][m]) / (max_val - min_val)
    return distances

###############################################################################
# 6) REAL-CODED GA OPERATORS: BLX-α CROSSOVER & POLYNOMIAL MUTATION
###############################################################################
def clamp_value(val, index):
    if index % 2 == 0:
        return min(max(val, XMIN), XMAX)
    else:
        return min(max(val, YMIN), YMAX)

def blend_crossover(pA, pB, alpha=ALPHA_BLX, rate=CROSS_RATE):
    if random.random() > rate:
        return pA[:], pB[:]
    c1, c2 = pA[:], pB[:]
    for i in range(len(pA)):
        x1, x2 = pA[i], pB[i]
        cmin = min(x1, x2)
        cmax = max(x1, x2)
        diff = cmax - cmin
        low = cmin - alpha * diff
        high = cmax + alpha * diff
        val1 = random.uniform(low, high)
        val2 = random.uniform(low, high)
        c1[i] = clamp_value(val1, i)
        c2[i] = clamp_value(val2, i)
    return c1, c2

def polynomial_mutation(chrom, rate=MUT_RATE, eta=ETA_MUT):
    for i in range(len(chrom)):
        if random.random() < rate:
            x = chrom[i]
            lower = XMIN if i % 2 == 0 else YMIN
            upper = XMAX if i % 2 == 0 else YMAX
            if abs(upper - lower) < 1e-14:
                continue
            delta1 = (x - lower) / (upper - lower)
            delta2 = (upper - x) / (upper - lower)
            rand = random.random()
            mut_pow = 1.0 / (eta + 1.0)
            if rand < 0.5:
                xy = 1.0 - delta1
                val = 2.0 * rand + (1.0 - 2.0 * rand) * (xy ** (eta + 1.0))
                deltaq = (val ** mut_pow) - 1.0
            else:
                xy = 1.0 - delta2
                val = 2.0 * (1.0 - rand) + 2.0 * (rand - 0.5) * (xy ** (eta + 1.0))
                deltaq = 1.0 - (val ** mut_pow)
            x = x + deltaq * (upper - lower)
            x = clamp_value(x, i)
            chrom[i] = x
    return chrom

def make_offspring(parentA, parentB):
    c1, c2 = blend_crossover(parentA, parentB, alpha=ALPHA_BLX, rate=CROSS_RATE)
    c1 = polynomial_mutation(c1, rate=MUT_RATE, eta=ETA_MUT)
    c2 = polynomial_mutation(c2, rate=MUT_RATE, eta=ETA_MUT)
    return c1, c2

###############################################################################
# 7) MAIN NSGA-II ALGORITHM
###############################################################################
def nsga2():
    population = [polygonal_chromosome() for _ in range(POP_SIZE)]
    best_cost_history = []
    best_coverage_history = []
    best_los_history = []

    for gen in range(MAX_GEN):
        obj_vals = [evaluate_cached(chrom) for chrom in population]

        offspring = []
        indices = list(range(POP_SIZE))
        random.shuffle(indices)
        for i in range(0, POP_SIZE - 1, 2):
            pA = population[indices[i]]
            pB = population[indices[i+1]]
            c1, c2 = make_offspring(pA, pB)
            offspring.append(c1)
            offspring.append(c2)
        if len(offspring) < POP_SIZE:
            offspring.append(population[indices[-1]])

        offspring_obj = [evaluate_cached(ch) for ch in offspring]
        combined = population + offspring
        combined_obj = obj_vals + offspring_obj

        ranks = fast_non_dominated_sort(combined_obj)
        new_population = []
        fronts = {}
        for i, rnk in enumerate(ranks):
            fronts.setdefault(rnk, []).append(i)

        for rnk in sorted(fronts.keys()):
            if len(new_population) + len(fronts[rnk]) > POP_SIZE:
                dist = crowding_distance(combined_obj, fronts[rnk])
                sorted_front = sorted(fronts[rnk], key=lambda idx: dist[idx], reverse=True)
                slots = POP_SIZE - len(new_population)
                new_population.extend([combined[idx] for idx in sorted_front[:slots]])
                break
            else:
                new_population.extend([combined[idx] for idx in fronts[rnk]])
        population = new_population

        best_front = fronts.get(0, [])
        if best_front:
            front_objs = [combined_obj[i] for i in best_front]
            best_cost = min(o[0] for o in front_objs)
            best_cov = max(-o[1] for o in front_objs)
            best_los = max(-o[2] for o in front_objs)

            best_cost_history.append(best_cost)
            best_coverage_history.append(best_cov)
            best_los_history.append(best_los)

            print(f"GEN {gen+1}/{MAX_GEN}: Best Cost={best_cost:.3f}, Best Coverage={best_cov:.3f}, Best LOS={best_los:.3f}")

    final_obj = [evaluate_cached(chrom) for chrom in population]
    final_ranks = fast_non_dominated_sort(final_obj)
    pareto_indices = [i for i, r in enumerate(final_ranks) if r == 0]
    pareto_solutions = [population[i] for i in pareto_indices]
    pareto_objs = [final_obj[i] for i in pareto_indices]

    return (pareto_solutions, pareto_objs, best_cost_history, best_coverage_history, best_los_history)
###############################################################################
# 8) MAIN EXECUTION & PLOTTING
###############################################################################
import folium
from shapely.geometry import MultiPoint

if __name__ == "__main__":
    pareto_sols, pareto_objs, cost_hist, cov_hist, los_hist = nsga2()
    if not pareto_sols:
        print("No Pareto solutions found for visualization.")
    else:
        best_sol = pareto_sols[0]

        best_gateway = (best_sol[0], best_sol[1])
        best_sensors = [(best_sol[i], best_sol[i+1]) for i in range(2, len(best_sol), 2)]

        # Create a Folium map with satellite tiles
        map_center = [(YMIN + YMAX) / 2, (XMIN + XMAX) / 2]
        m = folium.Map(location=map_center, zoom_start=13, tiles=None)
        folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri',
            name='Esri Satellite',
            overlay=False,
            control=True
        ).add_to(m)
        m.fit_bounds([[YMIN, XMIN], [YMAX, XMAX]])

        # Add gateway marker
        folium.Marker(
            location=[best_gateway[1], best_gateway[0]],
            popup='Best Gateway',
            icon=folium.Icon(color='red', icon='star')
        ).add_to(m)

        # Add sensor markers and coverage circles
        for i, sensor in enumerate(best_sensors):
            folium.CircleMarker(
                location=[sensor[1], sensor[0]],
                radius=5,
                color='green',
                fill=True,
                fill_color='green',
                popup=f"Sensor {i+1}"
            ).add_to(m)
            folium.Circle(
                location=[sensor[1], sensor[0]],
                radius=SENSOR_COVERAGE_RADIUS_M,  # radius in meters
                color='green',
                fill=True,
                fill_opacity=0.15,
                popup=f"Sensor {i+1} Coverage"
            ).add_to(m)

        # Visualize Polygon Formation
        polygon_coords = best_sensors + [best_gateway]
        if len(polygon_coords) >= 2:
            hull = MultiPoint(polygon_coords).convex_hull
            if hull.geom_type == 'Polygon':
                folium.PolyLine(
                    locations=[(point[1], point[0]) for point in hull.exterior.coords],
                    color='blue', weight=2, opacity=0.5,
                    popup="Sensor Polygon"
                ).add_to(m)
            elif hull.geom_type == 'LineString':
                folium.PolyLine(
                    locations=[(point[1], point[0]) for point in hull.coords],
                    color='blue', weight=2, opacity=0.7, dash_array='5',
                    popup="Sensor Polygon (Linear)"
                ).add_to(m)
            else:
                print("Warning: Convex hull is not a polygon or linestring for visualization.")

        # Draw lines connecting sensors to the gateway
        for sensor in best_sensors:
            folium.PolyLine(
                locations=[[sensor[1], sensor[0]], [best_gateway[1], best_gateway[0]]],
                color='orange',
                weight=2,
                opacity=0.7
            ).add_to(m)

        # Draw the domain boundary
        folium.Rectangle(
            bounds=[[YMIN, XMIN], [YMAX, XMAX]],
            color="blue",
            fill=True,
            fill_opacity=0.1,
            popup="Client Domain"
        ).add_to(m)

    


Found at centroid: (-117.2860371, 34.2475493)
Domain: X -117.3470103→-117.2422568, Y 34.2229549→34.2733466
DEM data loaded successfully from OpenTopography.
Sensors (excl. gateway): 7
Dynamically determined elevation threshold: 1443.50 meters
GEN 1/25: Best Cost=4489.143, Best Coverage=380.761, Best LOS=0.394
GEN 2/25: Best Cost=4489.143, Best Coverage=380.761, Best LOS=0.394
GEN 3/25: Best Cost=4487.048, Best Coverage=380.761, Best LOS=0.400
GEN 4/25: Best Cost=4487.048, Best Coverage=380.761, Best LOS=0.400
GEN 5/25: Best Cost=4487.048, Best Coverage=381.730, Best LOS=0.409
GEN 6/25: Best Cost=4487.048, Best Coverage=384.637, Best LOS=0.409
GEN 7/25: Best Cost=4487.048, Best Coverage=384.637, Best LOS=0.414
GEN 8/25: Best Cost=4484.868, Best Coverage=384.637, Best LOS=0.414
GEN 9/25: Best Cost=4484.868, Best Coverage=384.637, Best LOS=0.414
GEN 10/25: Best Cost=4484.868, Best Coverage=384.637, Best LOS=0.417
GEN 11/25: Best Cost=4484.445, Best Coverage=384.637, Best LOS=0.448
GEN 12/

In [4]:
m